Objective:  Track historical changes in master data.
Dimensions to Implement SCD-2
- Customers ( PK --> customer_id)
- Products (PK --> product_id)

Mandatory SCD Columns
- Business key (customer_id / product_id)
- Descriptive attributes
- effective_start_date
- effective_end_date
- is_Active


In [0]:



%sql
select * from main.information_schema.tables where table_schema="ecommerce"

In [0]:
%sql
use catalog main;
use schema ecommerce;

In [0]:
%sql
select * from customer;

In [0]:
%sql
describe customer

In [0]:
# note : create a test case for change data 

from pyspark.sql.functions import col
df_customer_src = spark.read.option("header",True).parquet("/Volumes/main/ecommerce/lakehouse_volumes/silver_dataset/customer/").filter(col("customer_id") == "f6aa4b1360cc4c6186561ebc2caf14bc").select("customer_id","customer_unique_id","customer_zip_code_prefix","customer_city","customer_state")

df_customer_src.write.mode("overwrite").saveAsTable("customer_src")

In [0]:
%sql
-- note : create a test case for change data 
-- note : update  test case for testing SCD2

update customer_src 
set 
  --customer_id = customer_id,
                    customer_city = "Varanasi",
                    customer_state = "UP"
where customer_id = 'f6aa4b1360cc4c6186561ebc2caf14bc';

In [0]:
%sql
-- ROW TO EXPIRE EXISTING RECORD
select tgt.customer_id,
       tgt.customer_unique_id,
       tgt.customer_zip_code_prefix,
       tgt.customer_city,
       tgt.customer_state,
       tgt.effective_start_date,
       current_date as effective_end_date,
       false as is_Active from customer tgt JOIN  customer_src src
on tgt.customer_id=src.customer_id
and tgt.is_Active = "true"
where (tgt.customer_zip_code_prefix <> src.customer_zip_code_prefix
  or tgt.customer_city <> src.customer_city
  or tgt.customer_state <> src.customer_state)

In [0]:
%sql
-- ROW TO INSERT NEW CURRENT RECORD
-- note : purposely appending (comma) to the  customer_id for new records , if numeric append minus sign 
select ("," || src.customer_id) as customer_id,
       src.customer_unique_id,
       src.customer_zip_code_prefix,
       src.customer_city,
       src.customer_state,
       current_date as effective_start_date,
       "9999-12-31" as effective_end_date,
       true as is_Active from customer tgt JOIN  customer_src src
on tgt.customer_id=src.customer_id
and tgt.is_Active = "true"
where (tgt.customer_zip_code_prefix <> src.customer_zip_code_prefix
  or tgt.customer_city <> src.customer_city
  or tgt.customer_state <> src.customer_state)

In [0]:
%sql
create or replace temp view cust_scd2_source
as
(
  select tgt.customer_id,
       tgt.customer_unique_id,
       tgt.customer_zip_code_prefix,
       tgt.customer_city,
       tgt.customer_state,
       --tgt.DESCRIPTIVE_ATTRIBUTES,
       tgt.effective_start_date,
       current_date as effective_end_date,
       false as is_Active from customer tgt JOIN  customer_src src
on tgt.customer_id=src.customer_id
and tgt.is_Active = "true"
where (tgt.customer_zip_code_prefix <> src.customer_zip_code_prefix
  or tgt.customer_city <> src.customer_city
  or tgt.customer_state <> src.customer_state)

UNION all

select ("," || src.customer_id) as customer_id,
       src.customer_unique_id,
       src.customer_zip_code_prefix,
       src.customer_city,
       src.customer_state,
       --src.DESCRIPTIVE_ATTRIBUTES,
       current_date as effective_start_date,
       "9999-12-31" as effective_end_date,
       true as is_Active 
  from customer tgt JOIN customer_src src
on tgt.customer_id=src.customer_id
and tgt.is_Active = "true"
where (tgt.customer_zip_code_prefix <> src.customer_zip_code_prefix
  or tgt.customer_city <> src.customer_city
  or tgt.customer_state <> src.customer_state)

)


In [0]:
%sql
-- note : in MATCHED AND condition
-- note : in MATCHED OR condition -- is not possible
-- note : puposly for new customer (comma) was added to make NOT MATCHED condtion true
-- note : instr --> LEFT and substr --> SUBSTRING
-- note : even if it run 2 times - there will be no change
-- note : AND LEFT(src.customer_id, 1) = ','  condition is not necessary 

MERGE INTO customer tgt
USING cust_scd2_source src
on tgt.customer_id = src.customer_id
and tgt.is_Active = true

WHEN MATCHED AND (src.is_Active == false) THEN
    UPDATE SET
    tgt.effective_end_date = '9999-12-31',
    tgt.is_Active = false

WHEN NOT MATCHED AND (src.is_Active == true) THEN
    INSERT (
      customer_id,
      customer_unique_id,
      customer_zip_code_prefix,
      customer_city,
      customer_state,
      --DESCRIPTIVE_ATTRIBUTES,
      effective_start_date,
      effective_end_date,
      is_Active
    )
    VALUES (
      substring(src.customer_id,2),
      src.customer_unique_id,
      src.customer_zip_code_prefix,
      src.customer_city,
      src.customer_state,
      --src.DESCRIPTIVE_ATTRIBUTES,
      current_date(),
      "9999-12-31",
      true
    );






In [0]:
%sql
select * from customer where customer_id = "f6aa4b1360cc4c6186561ebc2caf14bc"

In [0]:
%sql

select count(1) from customer ;

In [0]:
# note : writing to silver SCD 2 folder
spark.sql("select * from customer").write.mode("overwrite").parquet("/Volumes/main/ecommerce/lakehouse_volumes/silver_dataset/customer/cust_scd_2/")